The notebook resamples variables in .tif format to the resolution of the DEM (2m)

Importing relevant packages

In [1]:
from osgeo import gdal, gdalconst
import os

Setting up the directory

In [2]:
os.chdir(r"C:\Users\angelinkatula\Desktop\P8\climate_data")

The file that needs to be resamples. (in this example it is the average wind speed for future climate scenario RCP8.5, year 2700-2100

In [3]:
src_filename = 'wind_mean_future.tif'
src = gdal.Open(src_filename, gdalconst.GA_ReadOnly)
src_proj = src.GetProjection()
src_geotrans = src.GetGeoTransform()

Cheking its nodata value

In [4]:
NDV = src.GetRasterBand(1).GetNoDataValue()
NDV

1.0000000150474662e+30

The file which resolution is desired

In [5]:
# We want a section of source that matches this:
match_filename = r'C:\Users\angelinkatula\Desktop\P8\Variables\dem_elevation.tif'
match_ds = gdal.Open(match_filename, gdalconst.GA_ReadOnly)
match_proj = match_ds.GetProjection()
match_geotrans = match_ds.GetGeoTransform()
match_NDV = match_ds.GetRasterBand(1).GetNoDataValue()
wide = match_ds.RasterXSize
high = match_ds.RasterYSize
match_NDV

-9999.0

Resampling and saving output as .tif

In [6]:
# Output / destination
dst_filename = r'wind_m_future_resampled_2m.tif'
dst = gdal.GetDriverByName('GTiff').Create(dst_filename, wide, high, 1, gdalconst.GDT_Float32)
dst.SetGeoTransform( match_geotrans )
dst.SetProjection( match_proj)
dst.GetRasterBand(1).SetNoDataValue(match_NDV)

# Do the work
gdal.ReprojectImage(src, dst, src_proj, match_proj, gdalconst.GRA_NearestNeighbour)
dst.FlushCache()
del dst # Flush